In [0]:
from pyspark.sql.functions import current_timestamp, concat
from pyspark.sql.types import TimestampType, StringType

In [0]:
%run ../Utils/DBInstanceUtils

In [0]:
%run ../Utils/ProcessingUtils

In [0]:
instance = DBInstanceUtils('treinamento_formula1')
filename    = 'driversRaw'
extension   = 'delta'
file        = filename + '.' + extension

DBInstanceUtils.set_dbfs_filepath(instance, 'mnt',f'formula1/bronze/drivers', file)

dfColSchema = [
    "driver_id",
    "driver_reference",
    "number",
    "code",
    "dob",
    "nationality",
    "name"
]

In [0]:
dataframe = (spark.read
             .format('delta')
             .load(instance.get_dbfs_filepath())
             .drop('url'))

dataframe = (Refine(dataframe)
             .merge_cols('forename', 'surname', 'name')
             .renamecols(dfColSchema)
             .add_constcol('ingestionDate', TimestampType(),current_timestamp())
             .load())

In [0]:
filename = 'driversTrusted'
extension = 'delta'
file = filename + '.' + extension
DBInstanceUtils.set_dbfs_savepath(instance,'mnt','formula1/prata/drivers',file)

In [0]:
dataframe.write.mode('overwrite').save(instance.get_dbfs_savepath())